# ResNet50 - Demo
This notebook is a demo on a very small example subset of the ISIC Dataset. The dataset includes three classes, with 4 images in each train class and 2 images in each validation class. This small dataset is intended to generate quick (though necessarily not very accurate) results, and fit the storage limitations of a Github repo.
The ResNet50 model contained here is otherwise the same model I ran for this project using the entire ISIC dataset for classification, including pretrained Imagenet weights.

Running the code is as simple as opening this notebook in Google Colab and executing all the cells in order.

The code was adapted with modifications from here:

https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html 
License: BSD
Author: Sasank Chilamkurthy

In [2]:
# Installation of needed packages and mount drive
!pip install timm==0.4.12 yacs==0.1.8 adversarial-robustness-toolbox
!pip install -U PyYAML
# from google.colab import drive
# drive.mount('/content/drive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 376 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 69.8 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 596 kB 5.1 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Mounted at /content/drive


Resnet50 Pytorch

In [1]:
# Code adapted with modifications from https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html 
# License: BSD
# Author: Sasank Chilamkurthy

# Imports

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import sklearn
from timm.utils import accuracy, AverageMeter

cudnn.benchmark = True
plt.ion()   # interactive mode


ModuleNotFoundError: ignored

In [ ]:
# Load example dataset
url = f"https://raw.githubusercontent.com/jonvanveen/Medical-ViT-Classification-and-Robustness/tree/main/isic_example_data"
!wget --no-cache --backups=1 {url}

In [ ]:
# Transforms
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()#,
    ]),
    'val': transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor()#,
    ]),
}

# Train on ISIC Dataset
data_dir = '/content/drive/MyDrive/MLSP_Masters/ECE_697/data/isic/isic_org'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                  data_transforms[x])
                  for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=32, # batch size so small due to memory constraints
              shuffle=True, num_workers=2)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# AUC calculation 
from sklearn.metrics import roc_auc_score
def cal_auc(outputs, targets):
    outputs = outputs.squeeze()
    targets = targets.squeeze()

    if outputs.ndim == 1:
        # try-except block to avoid incorrect ValueError
        try:
            auc = roc_auc_score(targets, outputs)
        except ValueError:
            pass
    else:
        n_classes = outputs.shape[1]
        auc = 0
        for i in range(n_classes):
            try:
                label_auc = roc_auc_score(targets==i, outputs[:,i])
            except ValueError:
                pass
            auc += label_auc
        auc /= n_classes
    return auc

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=200):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    all_output, all_target = [], []

    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    #del outputs

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

                acc1, acc5 = accuracy(outputs,labels,topk=(1,5))

                all_output.append(outputs.detach().cpu().numpy())
                all_target.append(labels.detach().cpu().numpy())

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            time_elapsed = time.time() - since
            print("This epoch takes ", time_elapsed)

            all_output = np.concatenate(all_output)
            all_target = np.concatenate(all_target)
            auc = cal_auc(all_output, all_target)
            print(f"* AUC: {auc:.5f}")

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = models.resnet50(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, len(class_names))

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=200)

Epoch 0/199
----------
train Loss: 0.7109 Acc: 0.7618
This epoch takes  7501.6289303302765
* AUC: 0.83689


AttributeError: ignored

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.datasets as datasets
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data
from torchvision import transforms
import numpy as np
from art.estimators.classification import PyTorchClassifier
import rdnn.torch
import matplotlib.pyplot as plt

# Initialize the test datasets
test_dataset = rdnn.torch.CifarDataset(train=False, transform=transform_test)
x_test = test_dataset.get_x()
y_test = test_dataset.get_y()
print(len(test_dataset))
print("Loaded data.")

#################################################################################
# Load type of model -> don't forget to import if needed
model = Net()          
model.eval()
#################################################################################
path_to_checkpoint = "/cluster/work/math/robust-dnn/group5/krimmelm_2020-11-27_13.01.37_247514/ratio_0_6928571428571428"
optimizer, epoch, _ = rdnn.torch.load_model(path_to_checkpoint, model, from_shared_group_dir=False)
print("Loaded model.")
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)
classifier = PyTorchClassifier(
    model=model,
    clip_values=None,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
)
model.eval()

from art.attacks.evasion import AutoAttack
from art.attacks.evasion import AutoProjectedGradientDescent
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion import ProjectedGradientDescentPyTorch
from art.attacks.evasion import SquareAttack
from art.attacks.evasion import TargetedUniversalPerturbation
from art.attacks.evasion import UniversalPerturbation
from art.attacks.evasion import Wasserstein

eps = 0.03
eps_step = eps/3
norm = np.inf
ATTACK_fun = [
    AutoAttack(estimator=classifier, eps=eps, eps_step=eps_step, norm = norm),
    AutoProjectedGradientDescent(estimator=classifier, eps=eps, eps_step=eps_step, norm = norm),
    FastGradientMethod(estimator=classifier, eps=eps, eps_step=eps_step, norm = norm),
    ProjectedGradientDescentPyTorch(estimator=classifier, eps=eps, eps_step=eps_step, norm = norm),
    SquareAttack(estimator=classifier, eps=eps, norm = norm),
    # TargetedUniversalPerturbation(classifier = classifier, eps = eps, norm = norm ),
    # UniversalPerturbation(classifier = classifier, eps = eps, norm = norm ),
    Wasserstein(estimator=classifier, eps=eps, eps_step=eps_step, norm = str(norm))
]

for eval_attacks in ATTACK_fun:
    print(eval_attacks)
    x_test_adv = eval_attacks.generate(x_test)
    predictions = classifier.predict(x_test)
    accuracy = np.sum(np.argmax(predictions, axis=1) == y_test) / len(y_test)
    print("Accuracy on benign test examples: {}%".format(accuracy * 100))
    predictions_adv = classifier.predict(x_test_adv)
    accuracy_adv = np.sum(np.argmax(predictions_adv, axis=1) == y_test) / len(y_test)
    print("Accuracy on adversarial examples: {}%".format(accuracy_adv * 100))